# System Info

References: [thepythoncode.com](https://www.thepythoncode.com/article/get-hardware-system-information-python)

Open a shell an enter install psutil

```
pip3 install psutil
```

Let's make a function that converts large number of bytes into a scaled format (e.g in kilo, mega, giga, etc.):

In [198]:
import psutil
import platform
import pandas as pd
from datetime import datetime

def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor


## System Information

We gonna need platform module here:

### Platform information

In [199]:
def get_sysinfo():
    """
    Get some system information from platform module
    """
    # Get systeminfo
    uname = platform.uname()

    # Boot Time
    boot_time_timestamp = psutil.boot_time()
    boot_time_formated  = datetime.fromtimestamp(boot_time_timestamp)

    table_data = {
        'Systeminfo' : ['System','Node Name','Release','Version','Machine','Processor','Boot Time'],
        'Value': [uname.system,uname.node,uname.release,uname.version,uname.machine,uname.processor,boot_time_formated],
    }

    return pd.DataFrame(data = table_data)

### CPU information

In [200]:
def get_cpu_info():
    """
    Get some CPU information, such as total number of cores, usage, etc
    """
    cpufreq = psutil.cpu_freq()
    table_data = {
        'CPUs' : ["Physical cores","Total cores","Max Frequency","Min Frequency","Current Frequency","Total CPU Usage"],
        'Value': [psutil.cpu_count(logical=False),psutil.cpu_count(logical=True),f"{cpufreq.max:.0f}Mhz",f"{cpufreq.min:.0f}Mhz",f"{cpufreq.current:.0f}Mhz",f"{psutil.cpu_percent():.2f}%"],
    }

    return pd.DataFrame(data = table_data)

def get_cpu_info_det():
    """
    Get some CPU information, such as total number of cores, usage, etc
    """
    cpufreq = psutil.cpu_freq()
    cores = []
    usage = []
    # CPU Usage Per Core
    for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
        cores.append(f"Core {i}")
        usage.append(f"{percentage}%")

    table_data = {
        'Core' : cores,
        'Usage': usage,
    }
    return pd.DataFrame(table_data)

### Memory information

In [201]:
def get_memory_info():
    """
    Get memory information
    """
    # get the memory details
    svmem = psutil.virtual_memory()
    # get the swap memory details (if exists)
    swap = psutil.swap_memory()

    table_data = {
        'Memory'   : ["Total","Available","Used","Percentage"],
        'Physical' : [f"{get_size(svmem.total)}",f"{get_size(svmem.available)}",f"{get_size(svmem.used)}",f"{svmem.percent}%"],
        'Swap'     : [f"{get_size(swap.total)}",f"{get_size(swap.free)}",f"{get_size(swap.used)}",f"{swap.percent}%",],
    }    

    return pd.DataFrame(data = table_data)         

### Disk information

In [202]:
def get_disk_info():
    """
    Get disk information
    """
    partitions = psutil.disk_partitions()
    table_data = []
    for partition in partitions:
        try:
            partition_usage = psutil.disk_usage(partition.mountpoint)
        except PermissionError:
            # this can be catched due to the disk that
            # isn't ready
            continue

        thisdict = {  
            "Device"          : f"{partition.device}",
            "Mountpoint"      : f"{partition.mountpoint}",
            "File system type": f"{partition.fstype}",
            "Total Size"      : f"{get_size(partition_usage.total)}",
            "Used"            : f"{get_size(partition_usage.used)}",
            "Free"            : f"{get_size(partition_usage.free)}",
            "Percentage"      : f"{partition_usage.percent}%",
        }
        table_data.append(thisdict)

    return pd.DataFrame(data = table_data)    

### Network information

In [203]:
def get_net_info():
    """
    Get network information
    """
    # get all network interfaces (virtual and physical)
    table_data = []
    if_addrs = psutil.net_if_addrs()
    for interface_name, interface_addresses in if_addrs.items():
        for address in interface_addresses:
            if str(address.family) == 'AddressFamily.AF_INET':
                thisdict = {  
                    "Interface"   : f"{interface_name}",
                    "IP Address"  : f"{address.address}",
                    "Netmask"     : f"{address.netmask}",
                    "Broadcast IP": f"{address.broadcast}",
                }
            elif str(address.family) == 'AddressFamily.AF_PACKET':
                thisdict = {  
                    "Interface"    : f"{interface_name}",
                    "MAC Address"  : f"{address.address}",
                    "Netmask"      : f"{address.netmask}",
                    "Broadcast MAC": f"{address.broadcast}",
                }
        table_data.append(thisdict)

    return pd.DataFrame(data = table_data)

### Print all information

In [204]:
totallength = 110

# let's print system information
displaystr = "System Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_sysinfo())

# let's print CPU information
displaystr = "CPU Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_cpu_info())
display(get_cpu_info_det())

# let's print memory information
displaystr = "Memory Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_memory_info())

# let's print Disk information
displaystr = "Disk Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_disk_info())

# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read : {get_size(disk_io.read_bytes)}")
print(f"Total write: {get_size(disk_io.write_bytes)}")

# let's print Network information
displaystr = "Network Information"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))
display(get_net_info())

# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent    : {get_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")

displaystr = "END"
sidelengt = ((totallength - len(displaystr)) / 2)
print("="*int(sidelengt), displaystr, "="*int(sidelengt))

============================================== System Information ==============================================


,Systeminfo,Value
0,System,Darwin
1,Node Name,tinus-imac.home
2,Release,20.3.0
3,Version,Darwin Kernel Version 20.3.0: Thu Jan 21 00:07...
4,Machine,x86_64
5,Processor,i386
6,Boot Time,2021-05-21 07:37:04


=============================================== CPU Information ===============================================


,CPUs,Value
0,Physical cores,4
1,Total cores,8
2,Max Frequency,3600Mhz
3,Min Frequency,3600Mhz
4,Current Frequency,3600Mhz
5,Total CPU Usage,41.40%


,Core,Usage
0,Core 0,80.2%
1,Core 1,26.7%
2,Core 2,77.2%
3,Core 3,26.0%
4,Core 4,74.7%
5,Core 5,25.0%
6,Core 6,71.3%
7,Core 7,23.0%


...

=============================================== Disk Information ===============================================


,Device,Mountpoint,File system type,Total Size,Used,Free,Percentage
0,/dev/disk1s5s1,/,apfs,465.63GB,14.02GB,96.79GB,12.7%
1,/dev/disk1s4,/System/Volumes/VM,apfs,465.63GB,2.00GB,96.79GB,2.0%
2,/dev/disk1s2,/System/Volumes/Preboot,apfs,465.63GB,412.00MB,96.79GB,0.4%
3,/dev/disk1s6,/System/Volumes/Update,apfs,465.63GB,4.07MB,96.79GB,0.0%
4,/dev/disk1s1,/System/Volumes/Data,apfs,465.63GB,351.69GB,96.79GB,78.4%
5,/dev/disk2s2,/Volumes/Time Machine-Backups,hfs,931.19GB,505.90GB,425.29GB,54.3%


Total read : 24.91GB
Total write: 360.25GB
============================================= Network Information =============================================


,Interface,IP Address,Netmask,Broadcast IP
0,lo0,127.0.0.1,255.0.0.0,None
1,en1,192.168.1.112,255.255.255.0,192.168.1.255
2,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
3,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
4,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
5,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
6,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
7,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
8,bridge100,192.168.64.1,255.255.255.0,192.168.64.255
9,bridge100,192.168.64.1,255.255.255.0,192.168.64.255


Total Bytes Sent    : 66.93MB
Total Bytes Received: 193.98MB
===================================================== END =====================================================
